In [2]:
import jax.numpy as jnp
from jax import jit, random

In [1]:
import jax.numpy as jnp
from jax import jit, random, lax

def mandelbrot(c, max_iters=256, threshold=2):
    """
    Computes the number of iterations to determine if a complex number c
    is in the Mandelbrot set, considering a max_iters limit and escape threshold.
    """
    def body_fun(val):
        i, z = val
        return i + 1, z * z + c

    def cond_fun(val):
        i, z = val
        return (i < max_iters) & (jnp.abs(z) <= threshold)

    # Initialize the loop variables
    initial_state = (0, 0.0 + 0.0j)  # start with z = 0 + 0j
    result = lax.while_loop(cond_fun, body_fun, initial_state)

    # Final number of iterations and the last value of z
    num_iters, z_final = result

    # If z is within the threshold after max_iters, consider it inside the set
    return jnp.where(jnp.abs(z_final) <= threshold, max_iters, num_iters)

"""
    _, z_final = lax.while_loop(cond_fun, body_fun, (0, 0.0))
    return jnp.where(jnp.abs(z_final) <= threshold, max_iters, _)
"""


# JIT compile the function to optimize performance
mandelbrot_jit = jit(mandelbrot, static_argnums=(1, 2))

def estimate_mandelbrot_area(samples=100000, seed=0):
    """
    Estimates the area of the Mandelbrot set by sampling points in the complex plane,
    using Monte Carlo method.
    """
    # Generate random samples in the complex plane
    key = random.PRNGKey(seed)
    real = random.uniform(key, (samples,), minval=-2.0, maxval=1.0)
    imag = random.uniform(key, (samples,), minval=-1.5, maxval=1.5)
    c = real + 1j * imag

    # Apply the JIT-compiled Mandelbrot function to each point individually
    results = jnp.array([mandelbrot_jit(ci) for ci in c])
    #print(results[-50:-1])
    inside = jnp.mean(results == 256)  # Count how many are considered inside

    # Estimate area (scale by the area of the rectangle sampled)
    area = 3.0 * 3.0 * inside  # Area of the rectangle is 3 x 3
    return area

# Call the function and print the estimated area
area = estimate_mandelbrot_area()
print("Estimated area of the Mandelbrot set:", area)


Estimated area of the Mandelbrot set: 2.56563


In [2]:
from numba import cuda

# Check if a CUDA GPU is available
if cuda.is_available():
    print("CUDA is available. GPU:", cuda.gpus)
else:
    print("CUDA is not available.")


CUDA is available. GPU: <Managed Device 0>


In [3]:
from numba import cuda
print(cuda.gpus)


<Managed Device 0>


In [15]:
from numba import cuda
import numpy as np
import math

# Define the maximum number of iterations and the escape radius squared (4 for a radius of 2)
MAX_ITERS = 1024         # 1024 iterations bound
ESCAPE_RADIUS_SQ = 4.0

@cuda.jit
def mandelbrot_kernel(real, imag, output):
    """
    CUDA kernel to compute Mandelbrot membership.
    This kernel calculates whether each point in a grid of complex numbers
    belongs to the Mandelbrot set.
    """
    idx = cuda.grid(1)  # Compute the flat index within the array
    if idx < real.size:  # Check if index is within the bounds of the array
        c_real = real[idx]
        c_imag = imag[idx]
        z_real = 0.0
        z_imag = 0.0
        for n in range(MAX_ITERS):
            z_real2 = z_real * z_real
            z_imag2 = z_imag * z_imag
            if z_real2 + z_imag2 > ESCAPE_RADIUS_SQ:
                output[idx] = n
                break
            z_imag = 2.0 * z_real * z_imag + c_imag
            z_real = z_real2 - z_imag2 + c_real
        else:
            output[idx] = MAX_ITERS

def estimate_mandelbrot_area(samples=10_000_000, seed=0): # 10M seems to be the bound
    """
    Estimates the area of the Mandelbrot set by sampling points in the complex plane
    and using a Monte Carlo method.
    """
    print("Generating random samples...")
    np.random.seed(seed)
    real = np.random.uniform(-2.0, 1.0, samples).astype(np.float32)
    imag = np.random.uniform(-1.5, 1.5, samples).astype(np.float32)

    print("Allocating output array...")
    output = np.zeros_like(real, dtype=np.int32)

    # Calculate the number of threads per block and blocks per grid
    threads_per_block = 1024
    blocks_per_grid = math.ceil(samples / threads_per_block)
    print(f"Launching CUDA kernel with {blocks_per_grid} blocks and {threads_per_block} threads per block...")

    # Launch the kernel
    mandelbrot_kernel[blocks_per_grid, threads_per_block](real, imag, output)

    # Analyze the results to estimate the area
    print("Analyzing results...")
    inside = np.sum(output == MAX_ITERS) / samples
    area = 3.0 * 3.0 * inside  # Scale by the area of the rectangle sampled
    return area

# Call the function and print the estimated area
area = estimate_mandelbrot_area()
print(f"Estimated area of the Mandelbrot set: {area:.7f}")


Generating random samples...
Allocating output array...
Launching CUDA kernel with 9766 blocks and 1024 threads per block...


/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:888: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


Analyzing results...
Estimated area of the Mandelbrot set: 1.5100200


In [17]:
from numba import cuda
import numpy as np
import math

MAX_ITERS = 256
ESCAPE_RADIUS_SQ = 4.0

@cuda.jit
def mandelbrot_kernel(real, imag, output):
    idx = cuda.grid(1)
    if idx < real.size:
        c_real = real[idx]
        c_imag = imag[idx]
        z_real = 0.0
        z_imag = 0.0
        for n in range(MAX_ITERS):
            z_real2 = z_real * z_real
            z_imag2 = z_imag * z_imag
            if z_real2 + z_imag2 > ESCAPE_RADIUS_SQ:
                output[idx] = n
                break
            z_imag = 2.0 * z_real * z_imag + c_imag
            z_real = z_real2 - z_imag2 + c_real
        else:
            output[idx] = MAX_ITERS

def estimate_mandelbrot_area(samples=1000000, seed=0, n_resamples=1000):
    np.random.seed(seed)
    real = np.random.uniform(-2.0, 1.0, samples).astype(np.float32)
    imag = np.random.uniform(-1.5, 1.5, samples).astype(np.float32)

    output = np.zeros_like(real, dtype=np.int32)
    threads_per_block = 256
    blocks_per_grid = math.ceil(samples / threads_per_block)

    areas = np.zeros(n_resamples, dtype=np.float32)

    for i in range(n_resamples):
        # Resampling with replacement
        indices = np.random.choice(samples, samples, replace=True)
        sampled_real = real[indices]
        sampled_imag = imag[indices]

        # Launch kernel
        mandelbrot_kernel[blocks_per_grid, threads_per_block](sampled_real, sampled_imag, output)

        # Calculate area
        inside = np.sum(output == MAX_ITERS) / samples
        areas[i] = 3.0 * 3.0 * inside

    mean_area = np.mean(areas)
    std_deviation = np.std(areas)

    return mean_area, std_deviation

# Example usage
mean_area, uncertainty = estimate_mandelbrot_area()
print(f"Estimated area of the Mandelbrot set: {mean_area:.6f} ± {uncertainty:.6f}")


/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:888: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


Estimated area of the Mandelbrot set: 1.514171 ± 0.003348
